This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
import pydot
from IPython.display import SVG, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    InverseDynamicsController,
    LeafSystem,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    StateInterpolatorWithDiscreteDerivative,
)

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Controlling the iiwa with an allegro hand

The `MakeManipulationStation` code sets everything up assuming that you have an iiwa with a Schunk WSG gripper. What if you want to use the allegro hand instead? Then it probably makes sense implement the basic components of `MakeManipulationStation` yourself.

The simplest approach, which will be suitable for simulation, will be to use one `InverseDynamicsController` that treats the iiwa + Allegro as a single robot to control. If you want to run on the actual iiwa hardware, then we can do better (create two controllers, one for the iiwa which assumes an equivalent mass for the hand in a nominal fixed position + another controller for the hand), but that is not necessary to start.

In [ ]:
robot_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: allegro
    file: package://drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: allegro::hand_root
    X_PC:
        translation: [0, 0, 0.05]
        rotation: !Rpy { deg: [0, 0, 0]}
"""

environment_directives = """
directives:
- add_model:
    name: foam_brick
    file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
    default_free_body_pose:
        base_link:
            translation: [0.6, 0, 0]
- add_model:
    name: robot_table
    file: package://drake/examples/kuka_iiwa_arm/models/table/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: robot_table::link
    X_PC:
        translation: [0, 0, -0.7645]
- add_model:
    name: work_table
    file: package://drake/examples/kuka_iiwa_arm/models/table/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: work_table::link
    X_PC:
        translation: [0.75, 0, -0.7645]
"""

builder = DiagramBuilder()

time_step = 0.002
# Build one plant to model the world
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)
parser = Parser(plant)
parser.AddModelsFromString(robot_directives, ".dmd.yaml")
## Add any other models you want the robot to manipulate here.
parser.AddModelsFromString(environment_directives, ".dmd.yaml")
plant.Finalize()

# Build a second plant for the controller to use, which will ONLY have the iiwa
# and wsg. This one is *not* added to builder directly.
controller_plant = MultibodyPlant(time_step)
Parser(controller_plant).AddModelsFromString(robot_directives, ".dmd.yaml")
controller_plant.Finalize()
num_positions = controller_plant.num_positions()

# Add an inverse dynamics controller which uses the controller_plant.
inv_dynamics_controller = builder.AddSystem(
    InverseDynamicsController(
        controller_plant,
        kp=[100] * num_positions,
        ki=[1] * num_positions,
        kd=[20] * num_positions,
        has_reference_acceleration=False,
    )
)
inv_dynamics_controller.set_name("inv_dynamics_controller")
builder.Connect(
    inv_dynamics_controller.get_output_port_control(),
    plant.get_actuation_input_port(),
)
# Because we want this controller to use the estimated state of both the iiwa and the allegro (which are in two separate model instances), we need to mux those together.
iiwa = plant.GetModelInstanceByName("iiwa")
allegro = plant.GetModelInstanceByName("allegro")


class CombineIiwaAndAllegroState(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort(
            "iiwa_state", plant.num_multibody_states(iiwa)
        )
        self.DeclareVectorInputPort(
            "allegro_state", plant.num_multibody_states(allegro)
        )
        self.DeclareVectorOutputPort(
            "combined_state",
            plant.num_multibody_states(iiwa)
            + plant.num_multibody_states(allegro),
            self.CalcOutput,
        )

    def CalcOutput(self, context, output):
        iiwa_state = self.get_input_port(0).Eval(context)
        allegro_state = self.get_input_port(1).Eval(context)
        print(iiwa_state)
        print(allegro_state)
        # The order should should be [q_iiwa, q_allegro, v_iiwa, v_allegro]
        output.SetFromVector(
            np.concatenate(
                (
                    iiwa_state[: plant.num_positions(iiwa)],
                    allegro_state[: plant.num_positions(allegro)],
                    iiwa_state[plant.num_positions(iiwa) :],
                    allegro_state[plant.num_positions(allegro) :],
                )
            )
        )


combined_state = builder.AddSystem(CombineIiwaAndAllegroState())
builder.Connect(
    plant.get_state_output_port(iiwa), combined_state.get_input_port(0)
)
builder.Connect(
    plant.get_state_output_port(allegro), combined_state.get_input_port(1)
)
builder.Connect(
    combined_state.get_output_port(),
    inv_dynamics_controller.get_input_port_estimated_state(),
)

# Optional: If you want to only send positions, not also velocities (like the iiwa), then you can add this system:
desired_state_from_position = builder.AddSystem(
    StateInterpolatorWithDiscreteDerivative(
        controller_plant.num_positions(),
        time_step,
        suppress_initial_transient=True,
    )
)
builder.Connect(
    desired_state_from_position.get_output_port(),
    inv_dynamics_controller.get_input_port_desired_state(),
)

visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()

display(
    SVG(
        pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=1))[
            0
        ].create_svg()
    )
)

simulator = Simulator(diagram)
context = simulator.get_mutable_context()

q0 = combined_state.get_output_port().Eval(
    combined_state.GetMyContextFromRoot(context)
)[:num_positions]
desired_state_from_position.get_input_port().FixValue(
    desired_state_from_position.GetMyMutableContextFromRoot(context), q0
)

# Confirm that simulation works:
simulator.AdvanceTo(0.1)